<a href="https://colab.research.google.com/github/akaicomet/NJOITOpenDataCenter-Pension-Record/blob/master/NJOITOpenDataCenter_Pension_Record_ModelBuilding_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [79]:
!pip install pandas
!pip install tqdm
!pip install scikit-learn
!pip install sodapy
!pip install matplotlib
!pip install hyperopt
!pip install bayesian-optimization
!pip install tbvaccine 
!pip install MulticoreTSNE
!pip install -U -q PyDrive

#1. Copy python files from  GItHub repository to local

In [80]:
!git clone https://github.com/akaicomet/NJOITOpenDataCenter.git
!git clone https://github.com/akaicomet/akaicomet_Utility.git
!git clone https://github.com/akaicomet/akaicomet_Chart.git
!git clone https://github.com/akaicomet/akaicomet_ML.git

Cloning into 'NJOITOpenDataCenter'...
remote: Enumerating objects: 87, done.
remote: Counting objects: 100% (87/87), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 87 (delta 47), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (87/87), done.
Cloning into 'akaicomet_Utility'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 12 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (12/12), done.
Cloning into 'akaicomet_Chart'...
remote: Enumerating objects: 58, done.
remote: Counting objects: 100% (58/58), done.
remote: Compressing objects: 100% (46/46), done.
remote: Total 58 (delta 16), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (58/58), done.
Cloning into 'akaicomet_ML'...
remote: Enumerating objects: 20, done.
remote: Counting objects: 100% (20/20), done.
remote: Compressing objects: 100% (20/20), done.
remote: Total 171 (del

#2. Run .py files in local

In [0]:
%run -i /content/akaicomet_Utility/ErrorHandling.py

%run -i /content/NJOITOpenDataCenter/YourMoneyActivePensionMembers.py 
%run -i /content/NJOITOpenDataCenter/YourMoneyRetiredPensionMembers.py
%run -i /content/NJOITOpenDataCenter/YourMoneyPensionMembersMerge.py

%run -i /content/akaicomet_Chart/X-Ray_Scan.py
%run -i /content/akaicomet_Chart/akaicomet_Chart.py

%run -i /content/akaicomet_ML/ML_Init.py
%run -i /content/akaicomet_ML/ML_Auto.py
%run -i /content/akaicomet_ML/AutoFeatureEngineering.py


#3. Import libraries

In [0]:
import pandas as pd
import numpy as np
#import matplotlib.pyplot as plt
import random
from sodapy import Socrata
#from pandas.plotting import scatter_matrix
#from datetime import datetime
import seaborn as sb
from tqdm import tqdm
import warnings
from textwrap import wrap

In [0]:
from sklearn.externals import joblib
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


#4. YourMoney and the NJOIT Open Data Center 
##4.1 YourMoney Active Pension Members API

**Pension member data load from NJOIT** 

In [0]:
client = Socrata("data.nj.gov", None)
results = client.get("45bd-gwii", limit=100000000)#100000000

**Data Cleansing and Preparation for YourMoney Active Pension Members data**

In [0]:
ActivePension_df = ActivePensionDataWrangler(results)

##4.2 YourMoney Retired Pension Members API
**Pension payment member data load from NJOIT**

In [0]:
results = client.get("8up4-62p6",   limit=100000000)

**Data Cleansing and Preparation for YourMoney Retired Pension data**

In [0]:
RetiredPension_df_grped = RetiredPensionMembersDataWrangler(results)

**Merge YourMoney Active Pension Members data and YourMoney Retired Pension data**

In [0]:
ActivePension_df = PensionMembersMerge(ActivePension_df,RetiredPension_df_grped)
del results

## 5 Recheck  Correlation Variable Validation:

In [0]:
#x_cols2 = ['_20_year_status',	'all_employers_salary_amt',		'employer_freq_pensioned', 'total_months_qty',	'location_freq_pensioned',		'current_employer_salary_rollingamt', 	'service_months_qty',	'pension_freq_pensioned', 'pension_fund_name_enc', 'pension_group_name_enc' ]
x_cols2 = ['_20_year_status','all_employers_salary_amt','employer_freq_pensioned','total_months_qty','location_freq_pensioned','service_months_qty','pension_freq_pensioned','pension_fund_name_enc','pension_group_name_enc' ]
ActivePension_df[x_cols2].corr()

# 6 Load clfs Data from Google Drive
##  6.1 Join Table 

In [0]:
#Need to change 'id' everytime

downloaded = drive.CreateFile({'id':'1XRf0Zu7tYeCLRsqxxcuT9LhfyEVDhiPo'})
downloaded.GetContentFile('clfs_score612.pkl')

downloaded = drive.CreateFile({'id':'13BdyYzlrM4eD9HpUrmAign0cmLqrY7dl'})
downloaded.GetContentFile('clfs_score623.pkl')

In [0]:
clfs_score612 = joblib.load('clfs_score612.pkl')
clfs_score623 = joblib.load('clfs_score623.pkl')

In [0]:
df_clfs_score = pd.DataFrame.from_dict(clfs_score612,orient='index')

In [0]:
df_clfs_score2 = pd.DataFrame.from_dict(clfs_score623,orient='index')

In [0]:
df_clfs_score = pd.concat([df_clfs_score, df_clfs_score2], axis=1, join='inner')

## 6.2  Additional Field and Manupulation

In [0]:
df_clfs_score['classifier'] = [i.split('|')[0] for i  in df_clfs_score.index.tolist()]

In [0]:
df_clfs_score['std'] = df_clfs_score.std(axis=1)

In [0]:
df_clfs_score.columns = ['Training', 'Test1', 'Test2', 'Test3', 'classifier','std']

In [20]:
df_clfs_score.sort_values(['classifier','Training','std'], ascending=[True, False, True])

,Training,Test1,Test2,Test3,classifier,std
AdaBoostClassifier|PCA StandardScaler|PowerTransformer|,0.949143,0.930667,0.946000,0.922,AdaBoostClassifier,0.012826
AdaBoostClassifier|PCA MinMaxScaler|PowerTransformer|,0.948857,0.938000,0.948667,0.863,AdaBoostClassifier,0.041399
AdaBoostClassifier|Raw PowerTransformer|QuantileTransformer|,0.947143,0.937333,0.945333,0.937,AdaBoostClassifier,0.005291
AdaBoostClassifier|Raw StandardScaler|PowerTransformer|,0.946286,0.932000,0.944000,0.940,AdaBoostClassifier,0.006277
AdaBoostClassifier|Raw Normalizer|StandardScaler|,0.946286,0.947333,0.948667,0.933,AdaBoostClassifier,0.007280
AdaBoostClassifier|Raw Normalizer|PowerTransformer|,0.946000,0.938000,0.945333,0.937,AdaBoostClassifier,0.004740
AdaBoostClassifier|KPCA MinMaxScaler|QuantileTransformer|,0.945429,0.945333,0.947333,0.939,AdaBoostClassifier,0.003635
AdaBoostClassifier|Raw StandardScaler|QuantileTransformer|,0.944857,0.943333,0.946000,0.934,AdaBoostClassifier,0.005475
AdaBoostClassifier|PCA PowerTransformer|QuantileTransformer|,0.944000,0.939333,0.944667,0.938,AdaBoostClassifier,0.003328
AdaBoostClassifier|PCA StandardScaler|QuantileTransformer|,0.943429,0.942000,0.944000,0.939,AdaBoostClassifier,0.002236


In [15]:
set([i.split('|')[1].split(' ')[0] for i  in df_clfs_score.index.tolist()])

{'ICA', 'KPCA', 'MDS', 'PCA', 'Raw'}

In [16]:
set([i.split('|')[1].split(' ')[1] for i  in df_clfs_score.index.tolist()])

{'MinMaxScaler', 'Normalizer', 'PowerTransformer', 'StandardScaler'}

In [17]:
set([i.split('|')[2] for i  in df_clfs_score.index.tolist()])

{'Normalizer', 'PowerTransformer', 'QuantileTransformer', 'StandardScaler'}

## 6.3 Refine Redimensioning to Validate Model 

In [0]:
manager = Manager()
clfs3 = manager.dict()
clfs_score3 = manager.dict()

y_train = ActivePension_df.sample(n=1000, random_state=2)['IsPensionPaid'].as_matrix()
X_train = ActivePension_df[x_cols2].sample(n=1000, random_state=2)

X_train = X_train.astype(float)

scaling_def = {'MinMaxScaler': MinMaxScaler(),
               'QuantileTransformer_u':QuantileTransformer(output_distribution='uniform'),'QuantileTransformer_n':QuantileTransformer(output_distribution='normal'),
               'StandardScalerMTST': StandardScaler(with_mean=True, with_std=True),'StandardScalerMTSF': StandardScaler(with_mean=True, with_std=False),'StandardScalerMFST': StandardScaler(with_mean=False, with_std=True),'StandardScalerMFSF': StandardScaler(with_mean=False, with_std=False)
              }

redemensioning_def = {'Raw':None,
                      'PCA': PCA(n_components=0.99, svd_solver='full',whiten=False),'PCAW': PCA(n_components=0.99, svd_solver='full',whiten=True),
                      'ICA_P': FastICA(n_components=len(X_train.columns), algorithm='parallel'),'ICA_D': FastICA(n_components=len(X_train.columns), algorithm='deflation')
                     }

for clf in clfs:
  if clfs_score2[clf][0] > 0.95:
    scaling_def_temp = {i: scaling_def[i] for i in flatten([i.split(' ') for i in clf.split('|')])[1:] if i in scaling_def}
    redemensioning_def_temp = {i: redemensioning_def[i] for i in flatten([i.split(' ') for i in clf.split('|')])[1:] if i in redemensioning_def}
    for data in map(lambda x:Redimensioning2(x[1],x[0],redemensioning_def_temp), scaling3(X_train,scaling_def_temp)):
      for x in data:
        score = clfs[clf].score(x[0], y_train)
        if score > 0.95:                    
          p = Process(target=MLModelBuilding, args=(clf.split('|'))[0], x[1], x[0], y_train, 0.3, clfs3,[0.9,0.1],clfs_score3)
          p.start()
          p.join(180)
          p.terminate()          
          

In [0]:
flatten = lambda l: [item for sublist in l for item in sublist]
list(filter(lambda x: x != '', flatten([j[1:] for j in [i.split('|') for i in df_clfs_score.index.tolist()]])))

In [78]:
scaling_def = {'MinMaxScaler':MinMaxScaler(),'QuantileTransformer_u':QuantileTransformer(output_distribution='uniform'),'QuantileTransformer_n':QuantileTransformer(output_distribution='normal'),'StandardScalerMTST': StandardScaler(with_mean=True, with_std=True),'StandardScalerMTSF': StandardScaler(with_mean=True, with_std=False),'StandardScalerMFST': StandardScaler(with_mean=False, with_std=True),'StandardScalerMFSF': StandardScaler(with_mean=False, with_std=False)}

for clf in df_clfs_score.index.tolist():
  for i in clf.split(' ')[1:]:
    for j in i.split('|'):
      if j != '':
        scaling_def_temp = {s: scaling_def[s]  for s in scaling_def if j in s}

NameError: ignored

In [73]:
scaling_def = {'MinMaxScaler': MinMaxScaler(),
               'QuantileTransformer_u':QuantileTransformer(output_distribution='uniform'),'QuantileTransformer_n':QuantileTransformer(output_distribution='normal'),
               'StandardScalerMTST': StandardScaler(with_mean=True, with_std=True),'StandardScalerMTSF': StandardScaler(with_mean=True, with_std=False),'StandardScalerMFST': StandardScaler(with_mean=False, with_std=True),'StandardScalerMFSF': StandardScaler(with_mean=False, with_std=False)
              }

for clf in df_clfs_score.index.tolist():
  for i in clf.split(' ')[1:]:
    for j in i.split('|'):
      if j != '':
        scaling_def_temp = {s: scaling_def[s]  for s in scaling_def if j in s}


MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
Normalizer
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
StandardScaler
MinMaxScaler
PowerTransformer
MinMaxScaler
PowerTransformer
MinMaxScaler
PowerTransformer
MinMaxScaler
PowerTransformer
MinMaxScaler
PowerTransformer
MinMaxScaler
PowerTransformer
MinMaxScaler
PowerTransformer
MinMaxScaler
PowerTransformer
MinMaxScaler
PowerTransformer
MinMax

In [0]:
clfs32 = clfs3.copy()
clfs_score32 = clfs_score3.copy()

joblib.dump(clfs32, 'clfs624.pkl', compress=True)
joblib.dump(clfs_score32, 'clfs_score624.pkl', compress=True)

In [0]:
folder_id = '1wRbAVDSYm-NJsRMUqkRsreELnGaNM88R'
file = drive.CreateFile({'mimeType': 'content/clfs624.pkl','parents': [{'kind': 'drive#fileLink', 'id':folder_id}]})
file.SetContentFile('clfs624.pkl')
file.Upload() 
file = drive.CreateFile({'mimeType': 'content/clfs_score624.pkl','parents': [{'kind': 'drive#fileLink', 'id':folder_id}]})
file.SetContentFile('clfs_score624.pkl')
file.Upload() 

In [0]:
folder_id = '1wRbAVDSYm-NJsRMUqkRsreELnGaNM88R'
#file = drive.CreateFile({'clfs':'clfs.pkl', 'mimeType': 'content/clfs.pkl','parents': [{'kind': 'drive#fileLink', 'id':folder_id}]})
file = drive.CreateFile({'mimeType': 'content/clfs.pkl','parents': [{'kind': 'drive#fileLink', 'id':folder_id}]})
file.SetContentFile('clfs.pkl')
file.Upload() 